In [2]:
%load_ext autoreload
%autoreload 2
import nrrd
import numpy as np
import matplotlib.pyplot as plt
import ccf_streamlines.projection as ccfproj
import pathlib
import pandas as pd
import numpy as np
import os
import matplotlib.colors as mcolors
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.colors import LogNorm
import copy
import seaborn as sb
import nrrd
#from preprocessing_sequencing import preprocess_sequences as ps
from bg_atlasapi import BrainGlobeAtlas 
#from final_processing import final_processing_functions as fpf
import MAPseq_flatmap_functions as mff
import ast
import yaml
from scipy.ndimage import zoom
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import loading_functions as lf
import figure_formatting as ff
%matplotlib inline


First load parameters, barcode and sample dictionaries + initialise flatmap requirements

In [3]:
gen_parameters = lf.load_gen_params()
mice = gen_parameters['MICE']
saving_path = gen_parameters['fig_saving_path']
proj_path = gen_parameters['proj_path']
ff.set_font_params(gen_parameters)

In [ ]:
roi_dict, barcodes_dict, all_bcs_with_source, barcodes_no_source, sample_vol_and_regions = mff.make_dicts_of_mouse_3d_sample_rois(gen_parameters)
proj_top, bf_left_boundaries, bf_right_boundaries = mff.set_up_for_flatmaps(proj_path=gen_parameters['proj_path'])

Fig. 1g

In [ ]:
fig = plt.figure(figsize=(7.8, 2), facecolor="white",dpi=400)
mff.plot_indiv_projections(sample_vol_and_regions=sample_vol_and_regions, roi_dict=roi_dict, barcodes_dict=barcodes_dict, gen_parameters=gen_parameters, fig=fig, proj_top=proj_top, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries) 
fig.savefig(f"{saving_path}/fig_1_example_projections_updated.svg", format="svg", bbox_inches='tight', dpi=400)
fig.savefig(f"{saving_path}/fig_1_example_projections_updated.pdf", format="pdf", bbox_inches='tight', dpi=400) 

Fig. 2a

In [ ]:
normalised_bc_matrices, soma_coordinates_2d = mff.get_source_2d_and_normalised_matrices(roi_dict=roi_dict, gen_parameters=gen_parameters,  proj_top=proj_top, barcodes_no_source=barcodes_no_source)
to_look = mff.make_mean_3d(roi_dict=roi_dict, normalised_bc_matrices=normalised_bc_matrices, gen_parameters=gen_parameters, proj_top=proj_top)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
mff.plot_mean_flatmap(to_look=to_look, soma_coordinates_2d=soma_coordinates_2d, fig=fig, ax=ax, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries)
fig.savefig(f"{saving_path}/mean_soma_projection_densities_small.svg", format="svg", dpi=400)

Fig. 2d

In [7]:
mouse_dict_AP_source, mouse_dict_A1_coords = mff.get_AP_coords(
        gen_parameters=gen_parameters
    )
cortical_roi_dict, all_AP_dict, which_soma = mff.find_soma_AP_roi(
    mice=gen_parameters["MICE"],
    proj_path=gen_parameters["proj_path"],
    mouse_dict_AP_source=mouse_dict_AP_source,
    mouse_dict_A1_coords=mouse_dict_A1_coords,
    roi_dict=roi_dict,
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
mff.plot_AP_soma_cubelet_flatmap(gen_parameters=gen_parameters, fig=fig, ax=ax, roi_dict=roi_dict, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries, proj_top=proj_top)
fig.savefig(f"{saving_path}/mean_AP_soma_projection_all_cortex_small.svg", format="svg", dpi=400)

Extended data fig 2. a-b

In [ ]:
#nb. this takes a long time to run, since we generate the flatmap for each roi sequentially to avoid overlap in visualisation going from 3d to 2d
fig, axes = plt.subplots(2, 3, figsize=(7, 3.5))  
fig.subplots_adjust(wspace=0.30)
mff.plot_bulk_flatmaps_indiv(fig=fig, axes=axes, proj_top=proj_top, roi_dict=roi_dict, barcodes_dict=barcodes_dict, gen_parameters=gen_parameters, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries)
mff.plot_bulk_allen_anterograde(proj_top, fig, axes, bf_left_boundaries=bf_left_boundaries, bf_right_boundaries=bf_right_boundaries)
plt.tight_layout()
fig.savefig(f"{saving_path}/bulk_MAPseq_and_allen_flatmaps.svg", format="svg")
plt.show()